# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [47]:
print(requests.get(base_url, params=params))

<Response [400]>


In [28]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,71.65,76,56,22.77,PN,1723158193
1,1,albany,42.6001,-73.9662,65.26,75,99,3.00,US,1723157591
2,2,fortuna,40.5982,-124.1573,68.94,78,20,14.97,US,1723158196
3,3,grytviken,-54.2811,-36.5092,4.15,76,17,15.70,GS,1723158198
4,4,bredasdorp,-34.5322,20.0403,55.58,78,80,26.93,ZA,1723158200


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [12]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size="Humidity",         
    scale=1.0,
    color="Humidity"            
)

# Display the map plot
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [34]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_data_df = city_data_df.loc[
    (city_data_df["Max Temp"] >= 65) &
    (city_data_df["Max Temp"] <= 90) &
    (city_data_df["Humidity"] <= 60) &
    (city_data_df["Cloudiness"] <= 75)
]

# Drop any rows with null values
ideal_cities_data_df = ideal_cities_data_df.dropna()

# Display sample data
ideal_cities_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
13,13,yellowknife,62.4560,-114.3525,73.92,49,20,15.01,CA,1723158087
21,21,saryozek,44.3583,77.9753,71.10,28,0,5.53,KZ,1723158231
63,63,lethbridge,49.7000,-112.8186,69.91,43,40,0.00,CA,1723158313
67,67,prince george,53.9166,-122.7530,82.62,41,0,4.61,CA,1723158320
68,68,porto velho,-8.7619,-63.9039,86.05,54,4,0.00,BR,1723158053


### Step 3: Create a new DataFrame called `hotel_df`.

In [35]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
13,yellowknife,CA,62.4560,-114.3525,49,
21,saryozek,KZ,44.3583,77.9753,28,
63,lethbridge,CA,49.7000,-112.8186,43,
67,prince george,CA,53.9166,-122.7530,41,
68,porto velho,BR,-8.7619,-63.9039,54,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [46]:

# Set parameters to search for a hotel
radius = 10000
params = {
        "apiKey": geoapify_key,
        "categories":"accommodation.hotel",
        "limit":20
        }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"point:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places/"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
yellowknife - nearest hotel: No hotel found
saryozek - nearest hotel: No hotel found
lethbridge - nearest hotel: No hotel found
prince george - nearest hotel: No hotel found
porto velho - nearest hotel: No hotel found
douglas - nearest hotel: No hotel found
flin flon - nearest hotel: No hotel found
richards bay - nearest hotel: No hotel found
whitehorse - nearest hotel: No hotel found
port angeles - nearest hotel: No hotel found
san julian - nearest hotel: No hotel found
caiaponia - nearest hotel: No hotel found
sombrerete - nearest hotel: No hotel found
karratha - nearest hotel: No hotel found
zaragoza - nearest hotel: No hotel found
terrace - nearest hotel: No hotel found
lagunas - nearest hotel: No hotel found
nadi - nearest hotel: No hotel found
vernon - nearest hotel: No hotel found
poxoreo - nearest hotel: No hotel found
peace river - nearest hotel: No hotel found
avarua - nearest hotel: No hotel found
san pedro piedra gorda - nearest hotel: No hotel found
s

,City,Country,Lat,Lng,Humidity,Hotel Name
13,yellowknife,CA,62.4560,-114.3525,49,No hotel found
21,saryozek,KZ,44.3583,77.9753,28,No hotel found
63,lethbridge,CA,49.7000,-112.8186,43,No hotel found
67,prince george,CA,53.9166,-122.7530,41,No hotel found
68,porto velho,BR,-8.7619,-63.9039,54,No hotel found
74,douglas,US,41.3000,-96.1670,45,No hotel found
96,flin flon,CA,54.7682,-101.8650,40,No hotel found
99,richards bay,ZA,-28.7830,32.0377,41,No hotel found
122,whitehorse,CA,60.7161,-135.0538,32,No hotel found
125,port angeles,US,48.1181,-123.4307,54,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [51]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    size="Humidity", 
    color="Hotel Name",
    frame_width=800,
    frame_height=600,
    scale=1.5,  
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,Humidity)